In [1]:
#Upload drive folders
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
#Check the content of my notebooks folder
!ls drive/My\ Drive/Colab\ Notebooks/progetto2_visione/DeepLabV3_Plus-Tensorflow2.0/

1802.02611.pdf	     deeplabv3plus.png	inference.py   outputs	    train.ipynb
cityscapes_dict.pkl  gifs		last_epoch.h5  __pycache__  train.py
deeplab.py	     inference.ipynb	old_scripts    README.md    videos


In [4]:
#Set the actual directory
#!!!!!!!!!!!!!!!!!! Change it if you have an absolute path that is different !!!!!!!!!!!!!!!!!!!!!!!!
import os 
import sys
os.chdir("/content/drive/My Drive/Colab Notebooks/progetto2_visione/")
ROOT_DIR = os.getcwd()
print (ROOT_DIR)
os.chdir(os.path.join(ROOT_DIR, "DeepLabV3_Plus-Tensorflow2.0/"))

/content/drive/My Drive/Colab Notebooks/progetto2_visione


In [5]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from deeplab import DeepLabV3Plus
import tensorflow as tf
import cv2
from tqdm import tqdm
import os
import random
from glob import glob
import pickle
import time
import json
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.keras.utils import Sequence
from moviepy.editor import VideoFileClip, ImageSequenceClip
from tensorflow.keras.applications.resnet50 import preprocess_input

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3080192/45929032 bytes (6.7%)7290880/45929032 bytes (15.9%)11583488/45929032 bytes (25.2%)15704064/45929032 bytes (34.2%)20054016/45929032 bytes (43.7%)24109056/45929032 bytes (52.5%)27721728/45929032 bytes (60.4%)31932416/45929032 bytes (69.5%)36159488/45929032 bytes (78.7%)40361984/45929032 bytes (87.9%)44646400/45929032 bytes (97.2%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [6]:
h, w = 1080, 1920
with open('cityscapes_dict.pkl', 'rb') as f:
    id_to_color = pickle.load(f)['color_map']

# la strada è sette       
new_id_to_color = [0, 7, 11, 19, 20, 23, 24, 26, 27, 28, 32, 33] # quello che ci serve

# id_col = [x for i in id_to_color: i in new_id_to_color]

id_col = [id_to_color[x] for x in id_to_color if x in new_id_to_color]

print (id_col)



[(0, 0, 0), (128, 64, 128), (70, 70, 70), (250, 170, 30), (220, 220, 0), (70, 130, 180), (220, 20, 60), (0, 0, 142), (0, 0, 70), (0, 60, 100), (0, 0, 230), (119, 11, 32)]


In [8]:
model = DeepLabV3Plus(h, w, 34)
#model.load_weights('top_weights.h5')
model.load_weights('last_epoch.h5')

*** Building DeepLabv3Plus Network ***


W0620 21:59:29.818314 140545828964224 warnings.py:99] /usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '



ValueError: ignored

In [0]:
def pipeline(image, return_seg=False):
    global b
    alpha = 0.5
    dims = image.shape
    image = cv2.resize(image, (w, h))
    x = image.copy()
    z = model.predict(preprocess_input(np.expand_dims(x, axis=0)))
    z = np.squeeze(z)
    y = np.argmax(z, axis=2)
    
    img_color = image.copy()   
    for i in np.unique(y):
        if i in id_to_color:
            img_color[y==i] = id_to_color[i]

    if return_seg:
        cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)
        return img_color, y
    else:
        cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)      
        return img_color

In [0]:
def clean_segmentation(seg):
  
  #only moving objects
  mo_id_to_color = [24,25,26,27,28,29,32,33] # quello che ci serve
  
  new_seg = seg.copy()
  
  for i in range(new_seg.shape[0]):
    for j in range(new_seg.shape[1]):
      if new_seg[i][j] not in mo_id_to_color:
        new_seg[i][j] = 0
        
  return new_seg
           

In [0]:
def is_on_the_road(box, seg):
  x1, y1 = box[1], box[0]
  x2, y2 = box[3], box[2]
  width = abs(x2 - x1)
  height = abs(y2 - y1)
  
  THRESHOLD = 100
  
  road_pixel_counter = 0
  
  for i in range(y1, y2):
      for j in range(x1, x2):
        if seg[i][j] == 7 :
          road_pixel_counter = road_pixel_counter + 1
  print(road_pixel_counter)
          
  if road_pixel_counter > THRESHOLD:
    return True

  return False

In [0]:
def create_bounding_boxes(frame, seg):
  MINAREA = 1000
  MaxAREA = 1080*1920*0.80
  frame = img_to_array(frame)
  
  #clean_segmentation(seg)
  cleaned_seg = clean_segmentation(seg)
  
  map_labeled = measure.label(cleaned_seg, connectivity=1)
  for region in measure.regionprops(map_labeled):
    if region.area > MINAREA and region.area < MaxAREA :
        box = region.bbox
        p1 = (box[1], box[0])
        p2 = (box[3], box[2])
        
        if is_on_the_road(box, seg) == True:
          cv2.rectangle(frame, p1, p2, (77,255,9), 2)
        else:
          cv2.rectangle(frame, p1, p2, (255,255,255), 2)
        
        #vis_text(frame,label_names[seg_map[tuple(region.coords[0])]],(p1[0],p1[1]-10))
  
  return frame

In [0]:
#Video frames extraction
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from skimage import measure 

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#os.chdir("dataset/")
cap = cv2.VideoCapture(os.path.join(ROOT_DIR,'dataset/due.mp4'))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

counter = 0
  
# # Read until video is completed
while(cap.isOpened()):

  ret, frame = cap.read()
  counter = counter + 1
  if ret == True:
    if counter == 100 :
      counter = 0
      
      img_masked, seg = pipeline(frame, return_seg=True)
      
      img_masked = create_bounding_boxes(img_masked, seg)
      
      cv2_imshow(cv2.cvtColor(img_masked, cv2.COLOR_RGB2BGR))

#     # Press Q on keyboard to  exit


# # Break the loop
# else: 
#   print("ret is False")

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

In [0]:
#Video frames extraction
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from skimage import measure 

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#os.chdir("dataset/")
cap = cv2.VideoCapture(os.path.join(ROOT_DIR,'dataset/due.mp4'))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

counter = 0
  
# # Read until video is completed
while(cap.isOpened()):

  ret, frame = cap.read()
  counter = counter + 1
  if ret == True:
    if counter == 100 :
      counter = 0
      MINAREA = 1000
      MaxAREA = 1080*1920*0.80
      frame = img_to_array(frame)
      pipeline(frame, return_seg=False)
#       map_labeled = measure.label(seg, connectivity=1)
#       for region in measure.regionprops(map_labeled):
#           if region.area > MINAREA and region.area < MaxAREA :
#               box = region.bbox
#               p1 = (box[1], box[0])
#               p2 = (box[3], box[2])
#               cv2.rectangle(frame, p1, p2, (77,255,9), 2)
#     #           vis_text(frame,label_names[seg_map[tuple(region.coords[0])]],(p1[0],p1[1]-10))
#     s  cv2_imshow(cv2.cvtColor(seg, cv2.COLOR_RGB2BGR))


#     # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

# # Break the loop
# else: 
#   print("ret is False")

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

In [0]:
for image_dir in ['stuttgart_00', 'stuttgart_01', 'stuttgart_02']:
    os.mkdir(f'outputs/{image_dir}')
    image_list = os.listdir(image_dir)
    image_list.sort()
    print(f'{len(image_list)} frames found')
    for i in tqdm(range(len(image_list))):
        try:
            test = load_img(f'{image_dir}/{image_list[i]}')
            test = img_to_array(test)
            segmap = pipeline(test, video=False, fname=f'{image_list[i]}', folder=image_dir)
            if segmap == False:
                break
        except Exception as e:
            print(str(e))
    clip = ImageSequenceClip(sorted(glob(f'outputs/{image_dir}/*')), fps=18, load_images=True)
    clip.write_videofile(f'{image_dir}.mp4')